In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

/opt/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [18]:
import json

def search_for_questions(data):
    # 存储找到的所有 "qs_title" 内容
    titles = []
    title_count = 0  # 计数器

    def recursive_search(obj):
        nonlocal title_count  # 引用外部计数器
        if isinstance(obj, dict):
            for key, value in obj.items():
                if isinstance(value, (dict, list)):
                    recursive_search(value)  # 递归处理
                elif key.lower() == 'qs_title':  # 查找键名是 "qs_title"
                    titles.append(value)
                    title_count += 1  # 计数
        elif isinstance(obj, list):
            for item in obj:
                recursive_search(item)  # 递归处理列表中的每个项

    recursive_search(data)
    return titles, title_count

def load_data(file_name):
    """加载 JSON 文件并返回数据"""
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            data = json.load(file)  # 加载 JSON 数据
            print(f"文件 {file_name} 成功读取！")
            return data
    except FileNotFoundError:
        print(f"文件 {file_name} 未找到，请检查路径是否正确。")
        return None
    except json.JSONDecodeError:
        print(f"文件 {file_name} 格式错误，无法解析。")
        return None

# 文件名
file_name = 'tb_exam_question.json'

# 加载数据并搜索
loaded_data = load_data(file_name)

# 如果加载成功，则继续搜索
if loaded_data:
    found_titles, total_count = search_for_questions(loaded_data)

    # 打印找到的内容和数量
    if found_titles:
        print(f'找到的 "qs_title" 内容: {found_titles}')
        print(f'总共找到的 "qs_title" 数量: {total_count}')
    else:
        print('没有找到任何 "qs_title" 字段。')




文件 tb_exam_question.json 成功读取！
找到的 "qs_title" 内容: ['关于模拟万用表欧姆档使用，以下说法正确的是：', '关于模拟万用表直流电压档量程旁标注的1V和20K ，正确的说法是：', '关于模拟万用表表头下可调旋钮的说法，不正确的是：', '关于便携式数字万用表二极管测量档的显示数值，正确的是：', '便携式数字万用表直流电压档的输入阻抗是:', '便携式数字万用表直流电压档输入阻抗的指标是10M，关于其意义正确的是：', '用便携式数字万用表测量电阻时，如最高位显示“1”，说明：', 'SDM3055X-E型数字万用表的频率测量范围是：', '当接入一个 DC 电源信号，SDM3055X-E型数字万用表读数显示不正常，以下处理方式不正确的是：', '使用DM3055X-E型数字万用表测量电阻，不属于应用四线电阻测量法的条件是：', '如果按下电源开关，SDM3055X-E万用表仍然黑屏，没有任何显示，以下处理方式不正确的是：', '直流稳压电源的输出电压用以下哪种仪器测量最为便利？', '关于直流稳压电源液晶屏幕上显示的数值准确与否，说法正确的是：', 'SPD3000型直流稳压电源两路独立工作时额定输出电压和电流分别为：', '关于直流稳压电源“GND”公共接地端的说法，正确的是：', 'SDG1202X函数信号发生器的最大正弦波输出幅度是？', 'SDG2000X函数信号发生器方向键不能实现以下哪一项功能？', '不属于SDG2102X数字示波器多功能旋钮的功能是：', '不属于垂直控制区功能的是：', '控制波形在水平方向上显示周期的功能开关是：', '如果按下电源键示波器仍黑屏，无任何显示。不可以做的是：', '采集信号后，示波器屏幕上并未出现相应波形，应检查：', '使用示波器探头接入被测信号，发现测量的电压幅值比实际值大或者小，应检查：', '示波器屏幕有波形显示，但不能稳定下来，应检查：', '以下不属于电子元器件测量仪器的是：', '以下不属于信号分析类仪器的是：', '以下不属于示波器不同步的现象是：', '电阻器的性能指标主要包括：', '电阻的额定功率通常分为多少个等级：', '棕色环对应的电阻：', '金色色环代表的允许误差是:', '电位器的阻值变化规律通常有哪几种：', '用万用表检测电位器的

In [35]:
import jieba
import jieba.posseg as pseg
import gensim
from gensim.models import Word2Vec
import json

# 加载停用词
def load_stop_words(file_name):
    stop_words = set()
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            stop_words = {line.strip() for line in file if line.strip()}  # 去掉空行和换行符
        print(f"停用词文件 {file_name} 加载成功！")
    except FileNotFoundError:
        print(f"停用词文件 {file_name} 未找到，请检查文件路径。")
    return stop_words

# 预处理问题文本，进行分词和过滤
def preprocess_question(question, stop_words):
    words = pseg.cut(question)  # 使用 jieba 的词性标注功能
    knowledge_points = []

    for word, flag in words:
        # 过滤掉动词(v)、数词(m)、形容词(a)等词性，并排除停用词
        if flag not in ['v', 'm', 'a', 'd', 'p'] and word not in stop_words:  # v: 动词, m: 数词, a: 形容词, d: 副词, p: 介词
            if len(word) > 1:  # 只保留长度大于 1 的词
                knowledge_points.append(word)
    
    return knowledge_points

# 加载数据
def load_data(file_name):
    """加载 JSON 文件并返回数据"""
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            data = json.load(file)  # 加载 JSON 数据
            print(f"文件 {file_name} 成功读取！")
            return data
    except FileNotFoundError:
        print(f"文件 {file_name} 未找到，请检查路径是否正确。")
        return None
    except json.JSONDecodeError:
        print(f"文件 {file_name} 格式错误，无法解析。")
        return None

# 训练 Word2Vec 模型
def train_word2vec_model(texts, vector_size=100, window=5, min_count=1):
    """根据文本训练 Word2Vec 模型"""
    model = Word2Vec(sentences=texts, vector_size=vector_size, window=window, min_count=min_count, workers=4)
    model.save("word2vec.model")  # 保存模型
    return model

# 使用 Word2Vec 提取关键词
def extract_keywords_word2vec(model, words, top_n=10):
    """使用 Word2Vec 模型提取关键词"""
    word_scores = []
    for word in words:
        if word in model.wv:
            # 计算每个词与其他词的相似度
            similar_words = model.wv.most_similar(word, topn=top_n)
            word_scores.extend([w for w, _ in similar_words])  # 收集相似的词
    
    # 去重
    return set(word_scores)

# 文件名
file_name = 'tb_exam_question.json'
stop_words_file = 'stop_words.txt'  # 停用词文件

# 加载停用词
stop_words = load_stop_words(stop_words_file)

# 加载数据
loaded_data = load_data(file_name)

if loaded_data:
    # 提取所有 "qs_title" 字段并进行分词
    questions = [item.get('qs_title', '') for item in loaded_data]
    
    # 处理所有问题文本，提取知识点
    processed_questions = [preprocess_question(q, stop_words) for q in questions]
    
    # 训练 Word2Vec 模型
    model = train_word2vec_model(processed_questions)

    # 提取中心词（基于 Word2Vec 模型）
    all_keywords = []
    for question_keywords in processed_questions:
        for keyword in question_keywords:
            if keyword in model.wv:
                all_keywords.append(keyword)

    # 使用 Word2Vec 提取相关关键词
    final_keywords = extract_keywords_word2vec(model, all_keywords, top_n=10)

    # 输出结果
    print("最终提取的知识点:", final_keywords)
    print(f"知识点的数量: {len(final_keywords)}")


停用词文件 stop_words.txt 加载成功！
文件 tb_exam_question.json 成功读取！
最终提取的知识点: {'电压表', '器能', '直流', '容值', '电磁干扰', '交流电', '波段', '低电压', '信号源', '框图', '互感器', 'SDG1202X', '____', '压敏电阻', '过阻尼', '高通', '负荷', '电动式', '三极管', '单级', '电感', '压档', '用途', '角度', '滤波器', '直流电', '瓷片', '压源', 'RP', '监测', '电容器', 'SPD3000', '流向', '尺寸', '布线', '自激振荡', '相位差', '万用表', '安全电压', '伏安', '质量', '关系', '交流电路', '胸外', '稳压电源', '磁通', '容量', '绝对误差', '氖管', '谐振', 'Vi1', 'CA', '基极', '电流强度', '间接测量', '电解电容', '镇流器', 'SDG2102X', '量限', '内阻', '临界阻尼', '端电压', '元器件', '测量误差', '变化', '速度', '衰减系数', '检流计', '电位器', '灵敏度', '电感器', '导通', '比例', '测量范围', '互感', '供电系统', '端口', '输入阻抗', 'Db', 'SPD32000', '幅度', '电阻器', '变化规律', '偏置', '二端网络', 'BNC', '直流电源', 'output', '双电源', '稳态', '互感系数', '信号', '频率特性', 'DM3055X', '钳表', '热敏电阻', '低通滤波器', '交流', '增益', '额定功率', '负载电阻', '比率', '工作电压', '波形', '板结构', '时间常数', '正弦波', '功率', '幅频特性', '功率表', '标称', '焊点', '相频特性', 'RLC', '幅值', '相电流', '变压器', '振荡器', '黑色', '方波', 'Cursors', '积分电路', '串并联', '共地', '输入电阻', '相线', '电感线圈', '端相', '方向键', '示波器', '截止频率', '电阻值'

In [2]:
import json
import pandas as pd
from collections import defaultdict
from tqdm import tqdm  # 用于显示进度条

def generate_q_matrix(json_path, output_path=None):
    """
    自动读取JSON文件并生成Q矩阵
    
    参数:
        json_path: JSON文件路径
        output_path: 可选，Q矩阵输出路径(.csv或.xlsx)
    
    返回:
        Q矩阵(DataFrame格式)
    """
    # 1. 读取JSON文件
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"成功读取JSON文件: {json_path}")
    except Exception as e:
        print(f"读取JSON文件失败: {e}")
        return None
    
    # 2. 提取题目和知识点数据
    q_matrix_data = defaultdict(dict)
    knowledge_points = set()
    
    print("开始处理题目数据...")
    for item in tqdm(data, desc="处理进度"):
        question_id = item.get('qs_id', str(hash(str(item))))  # 使用qs_id或生成唯一标识
        question_text = item.get('qs_title', '')
        
        # 使用您提供的预处理函数提取知识点
        processed_kps = preprocess_question(question_text, stop_words)
        
        # 记录知识点
        for kp in processed_kps:
            q_matrix_data[question_id][kp] = 1
            knowledge_points.add(kp)
    
    # 3. 构建Q矩阵
    knowledge_points = sorted(list(knowledge_points))
    q_matrix = pd.DataFrame(
        index=q_matrix_data.keys(),
        columns=knowledge_points,
        dtype=int
    ).fillna(0)
    
    # 填充矩阵
    for q_id, kps in q_matrix_data.items():
        for kp in kps:
            q_matrix.at[q_id, kp] = 1
    
    # 4. 可选: 保存结果
    if output_path:
        try:
            if output_path.endswith('.csv'):
                q_matrix.to_csv(output_path)
                print(f"Q矩阵已保存为CSV文件: {output_path}")
            elif output_path.endswith('.xlsx'):
                q_matrix.to_excel(output_path)
                print(f"Q矩阵已保存为Excel文件: {output_path}")
            else:
                print("未识别的输出格式，请使用.csv或.xlsx后缀")
        except Exception as e:
            print(f"保存文件时出错: {e}")
    
    return q_matrix

# 使用示例（整合到您现有代码中）
if __name__ == "__main__":
    # 您的现有配置
    file_name = 'tb_exam_question.json'
    stop_words_file = 'stop_words.txt'
    
    # 加载停用词
    stop_words = load_stop_words(stop_words_file)
    
    # 生成Q矩阵
    q_matrix = generate_q_matrix(file_name, 'q_matrix_output.csv')
    
    if q_matrix is not None:
        print("\n生成的Q矩阵预览:")
        print(q_matrix.head())
        print(f"\nQ矩阵维度: {q_matrix.shape} (题目数 × 知识点数)")


NameError: name 'load_stop_words' is not defined

In [45]:
import jieba
import jieba.posseg as pseg
import gensim
from gensim.models import Word2Vec
import json

# 加载停用词
def load_stop_words(file_name):
    stop_words = set()
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            stop_words = {line.strip() for line in file if line.strip()}  # 去掉空行和换行符
        print(f"停用词文件 {file_name} 加载成功！")
    except FileNotFoundError:
        print(f"停用词文件 {file_name} 未找到，请检查文件路径。")
    return stop_words

# 预处理问题文本，进行分词和过滤
def preprocess_question(question, stop_words):
    words = pseg.cut(question)  # 使用 jieba 的词性标注功能
    knowledge_points = []

    for word, flag in words:
        # 过滤掉动词(v)、数词(m)、形容词(a)等词性，并排除停用词
        if flag not in ['v', 'm', 'a', 'd', 'p'] and word not in stop_words:  # v: 动词, m: 数词, a: 形容词, d: 副词, p: 介词
            if len(word) > 1:  # 只保留长度大于 1 的词
                knowledge_points.append(word)
    
    return knowledge_points

# 加载数据
def load_data(file_name):
    """加载 JSON 文件并返回数据"""
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            data = json.load(file)  # 加载 JSON 数据
            print(f"文件 {file_name} 成功读取！")
            return data
    except FileNotFoundError:
        print(f"文件 {file_name} 未找到，请检查路径是否正确。")
        return None
    except json.JSONDecodeError:
        print(f"文件 {file_name} 格式错误，无法解析。")
        return None

# 训练 Word2Vec 模型
def train_word2vec_model(texts, vector_size=100, window=5, min_count=1):
    """根据文本训练 Word2Vec 模型"""
    model = Word2Vec(sentences=texts, vector_size=vector_size, window=window, min_count=min_count, workers=4)
    model.save("word2vec.model")  # 保存模型
    return model

# 使用 Word2Vec 提取关键词
def extract_keywords_word2vec(model, words, top_n=10):
    """使用 Word2Vec 模型提取关键词"""
    word_scores = []
    for word in words:
        if word in model.wv:
            # 计算每个词与其他词的相似度
            similar_words = model.wv.most_similar(word, topn=top_n)
            word_scores.extend([w for w, _ in similar_words])  # 收集相似的词
    
    # 去重
    return set(word_scores)

# 创建 Q 矩阵：将问题的知识点与知识点列表进行对比，生成二值矩阵
def create_q_matrix(questions, knowledge_points, word_to_idx):
    q_matrix = []
    for question in questions:
        # 将问题拆分成单词
        words = preprocess_question(question, stop_words)
        # 创建与知识点匹配的二值向量
        q_vector = [0] * len(knowledge_points)
        for word in words:
            if word in word_to_idx:
                # 检查该单词是否属于知识点
                if word in knowledge_points:
                    q_vector[knowledge_points.index(word)] = 1
        q_matrix.append(q_vector)
    return q_matrix

# 文件名
file_name = 'tb_exam_question.json'
stop_words_file = 'stop_words.txt'  # 停用词文件

# 加载停用词
stop_words = load_stop_words(stop_words_file)

# 加载数据
loaded_data = load_data(file_name)

if loaded_data:
    # 提取所有 "qs_title" 字段并进行分词
    questions = [item.get('qs_title', '') for item in loaded_data]
    
    # 处理所有问题文本，提取知识点
    processed_questions = [preprocess_question(q, stop_words) for q in questions]
    
    # 训练 Word2Vec 模型
    model = train_word2vec_model(processed_questions)

    # 提取中心词（基于 Word2Vec 模型）
    all_keywords = []
    for question_keywords in processed_questions:
        for keyword in question_keywords:
            if keyword in model.wv:
                all_keywords.append(keyword)

    # 使用 Word2Vec 提取相关关键词
    final_keywords = extract_keywords_word2vec(model, all_keywords, top_n=10)

    # 输出结果
    print("最终提取的知识点:", final_keywords)
    print(f"知识点的数量: {len(final_keywords)}")


停用词文件 stop_words.txt 加载成功！
文件 tb_exam_question.json 成功读取！
最终提取的知识点: {'电压表', '器能', '直流', '容值', '电磁干扰', '交流电', '波段', '低电压', '信号源', '框图', '互感器', 'SDG1202X', '____', '压敏电阻', '过阻尼', '高通', '负荷', '电动式', '三极管', '单级', '电感', '压档', '用途', '角度', '滤波器', '直流电', '瓷片', '压源', 'RP', '监测', '电容器', 'SPD3000', '流向', '尺寸', '布线', '自激振荡', '相位差', '万用表', '安全电压', '伏安', '质量', '关系', '交流电路', '胸外', '稳压电源', '磁通', '容量', '绝对误差', '氖管', '谐振', 'Vi1', 'CA', '基极', '电流强度', '间接测量', '电解电容', '镇流器', 'SDG2102X', '量限', '内阻', '临界阻尼', '端电压', '元器件', '测量误差', '变化', '速度', '衰减系数', '检流计', '电位器', '灵敏度', '电感器', '导通', '比例', '测量范围', '互感', '供电系统', '端口', '输入阻抗', 'Db', 'SPD32000', '幅度', '电阻器', '变化规律', '偏置', '二端网络', 'BNC', '直流电源', 'output', '双电源', '稳态', '互感系数', '信号', '频率特性', 'DM3055X', '钳表', '热敏电阻', '低通滤波器', '交流', '增益', '额定功率', '负载电阻', '比率', '工作电压', '波形', '板结构', '时间常数', '正弦波', '功率', '幅频特性', '功率表', '标称', '焊点', '相频特性', 'RLC', '幅值', '相电流', '变压器', '振荡器', '黑色', '方波', 'Cursors', '积分电路', '串并联', '共地', '输入电阻', '相线', '电感线圈', '端相', '方向键', '示波器', '截止频率', '电阻值'

In [44]:
# 加载数据
loaded_data = load_data(file_name)

if loaded_data:
    # 提取所有 "qs_title" 字段并进行分词
    questions = [item.get('qs_title', '') for item in loaded_data]
    
    # 处理所有问题文本，提取知识点
    processed_questions = [preprocess_question(q, stop_words) for q in questions]
    
    # 训练 Word2Vec 模型
    model = train_word2vec_model(processed_questions)

    # 提取中心词（基于 Word2Vec 模型）
    all_keywords = []
    for question_keywords in processed_questions:
        for keyword in question_keywords:
            if keyword in model.wv:
                all_keywords.append(keyword)

    # 使用 Word2Vec 提取相关关键词
    final_keywords = extract_keywords_word2vec(model, all_keywords, top_n=10)
    
    # 将最终提取的知识点作为知识点列表
    knowledge_points = list(final_keywords)
    
    # 创建 Q 矩阵
    q_matrix = create_q_matrix(questions, knowledge_points, model.wv.key_to_index)
    
    # 输出 Q 矩阵的部分内容
    print(f"Q 矩阵的前几行：")
    for row in q_matrix[:20]:  # 输出前五行
        print(row)


文件 tb_exam_question.json 成功读取！
Q 矩阵的前几行：
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [46]:
import jieba
import jieba.posseg as pseg
import gensim
from gensim.models import Word2Vec
import json
import pandas as pd

# 加载停用词
def load_stop_words(file_name):
    stop_words = set()
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            stop_words = {line.strip() for line in file if line.strip()}  # 去掉空行和换行符
        print(f"停用词文件 {file_name} 加载成功！")
    except FileNotFoundError:
        print(f"停用词文件 {file_name} 未找到，请检查文件路径。")
    return stop_words

# 预处理问题文本，进行分词和过滤
def preprocess_question(question, stop_words):
    words = pseg.cut(question)  # 使用 jieba 的词性标注功能
    knowledge_points = []

    for word, flag in words:
        # 过滤掉动词(v)、数词(m)、形容词(a)等词性，并排除停用词
        if flag not in ['v', 'm', 'a', 'd', 'p'] and word not in stop_words:  # v: 动词, m: 数词, a: 形容词, d: 副词, p: 介词
            if len(word) > 1:  # 只保留长度大于 1 的词
                knowledge_points.append(word)
    
    return knowledge_points

# 加载数据
def load_data(file_name):
    """加载 JSON 文件并返回数据"""
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            data = json.load(file)  # 加载 JSON 数据
            print(f"文件 {file_name} 成功读取！")
            return data
    except FileNotFoundError:
        print(f"文件 {file_name} 未找到，请检查路径是否正确。")
        return None
    except json.JSONDecodeError:
        print(f"文件 {file_name} 格式错误，无法解析。")
        return None

# 训练 Word2Vec 模型
def train_word2vec_model(texts, vector_size=100, window=5, min_count=1):
    """根据文本训练 Word2Vec 模型"""
    model = Word2Vec(sentences=texts, vector_size=vector_size, window=window, min_count=min_count, workers=4)
    model.save("word2vec.model")  # 保存模型
    return model

# 使用 Word2Vec 提取关键词
def extract_keywords_word2vec(model, words, top_n=10):
    """使用 Word2Vec 模型提取关键词"""
    word_scores = []
    for word in words:
        if word in model.wv:
            # 计算每个词与其他词的相似度
            similar_words = model.wv.most_similar(word, topn=top_n)
            word_scores.extend([w for w, _ in similar_words])  # 收集相似的词
    
    # 去重
    return set(word_scores)

# 创建 Q 矩阵：将问题的知识点与知识点列表进行对比，生成二值矩阵
def create_q_matrix(questions, knowledge_points, word_to_idx):
    q_matrix = []
    for question in questions:
        # 将问题拆分成单词
        words = preprocess_question(question, stop_words)
        # 创建与知识点匹配的二值向量
        q_vector = [0] * len(knowledge_points)
        for word in words:
            if word in word_to_idx:
                # 检查该单词是否属于知识点
                if word in knowledge_points:
                    q_vector[knowledge_points.index(word)] = 1
        q_matrix.append(q_vector)
    return q_matrix

# 文件名
file_name = 'tb_exam_question.json'
stop_words_file = 'stop_words.txt'  # 停用词文件

# 加载停用词
stop_words = load_stop_words(stop_words_file)

# 加载数据
loaded_data = load_data(file_name)

if loaded_data:
    # 提取所有 "qs_title" 字段并进行分词
    questions = [item.get('qs_title', '') for item in loaded_data]
    
    # 处理所有问题文本，提取知识点
    processed_questions = [preprocess_question(q, stop_words) for q in questions]
    
    # 训练 Word2Vec 模型
    model = train_word2vec_model(processed_questions)

    # 提取中心词（基于 Word2Vec 模型）
    all_keywords = []
    for question_keywords in processed_questions:
        for keyword in question_keywords:
            if keyword in model.wv:
                all_keywords.append(keyword)

    # 使用 Word2Vec 提取相关关键词
    final_keywords = extract_keywords_word2vec(model, all_keywords, top_n=10)

    # 输出结果
    print("最终提取的知识点:", final_keywords)
    print(f"知识点的数量: {len(final_keywords)}")
    
    # 将最终提取的知识点作为知识点列表
    knowledge_points = list(final_keywords)
    
    # 创建 Q 矩阵
    q_matrix = create_q_matrix(questions, knowledge_points, model.wv.key_to_index)
    
    # 将 Q 矩阵和知识点转换为 pandas DataFrame
    df = pd.DataFrame(q_matrix, columns=knowledge_points)

    # 输出到 Excel 文件
    output_file = 'knowledge_point_matrix.xlsx'
    df.to_excel(output_file, index=False, sheet_name='Knowledge Points Matrix')

    print(f"Q 矩阵已成功保存到 {output_file}")


停用词文件 stop_words.txt 加载成功！
文件 tb_exam_question.json 成功读取！
最终提取的知识点: {'电压表', '器能', '直流', '容值', '电磁干扰', '交流电', '波段', '低电压', '信号源', '框图', '互感器', 'SDG1202X', '____', '压敏电阻', '过阻尼', '高通', '负荷', '电动式', '三极管', '单级', '电感', '压档', '用途', '角度', '滤波器', '直流电', '瓷片', '压源', 'RP', '监测', '电容器', 'SPD3000', '流向', '尺寸', '布线', '自激振荡', '相位差', '万用表', '安全电压', '伏安', '质量', '关系', '交流电路', '胸外', '稳压电源', '磁通', '容量', '绝对误差', '氖管', '谐振', 'Vi1', 'CA', '基极', '电流强度', '间接测量', '电解电容', '镇流器', 'SDG2102X', '量限', '内阻', '临界阻尼', '端电压', '元器件', '测量误差', '变化', '速度', '衰减系数', '检流计', '电位器', '灵敏度', '电感器', '导通', '比例', '测量范围', '互感', '供电系统', '端口', '输入阻抗', 'Db', 'SPD32000', '幅度', '电阻器', '变化规律', '偏置', '二端网络', 'BNC', '直流电源', 'output', '双电源', '稳态', '互感系数', '信号', '频率特性', 'DM3055X', '钳表', '热敏电阻', '低通滤波器', '交流', '增益', '额定功率', '负载电阻', '比率', '工作电压', '波形', '板结构', '时间常数', '正弦波', '功率', '幅频特性', '功率表', '标称', '焊点', '相频特性', 'RLC', '幅值', '相电流', '变压器', '振荡器', '黑色', '方波', 'Cursors', '积分电路', '串并联', '共地', '输入电阻', '相线', '电感线圈', '端相', '方向键', '示波器', '截止频率', '电阻值'